<a href="https://colab.research.google.com/github/Khaarl/VIZDOOM1/blob/03/VISDOOMGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
+-----------------------------------------------------------------------------+
| |       |\                                           -~ /     \  /          |
|~~__     | \                                         | \/       /\          /|
|    --   |  \                                        | / \    /    \     /   |
|      |~_|   \                                   \___|/    \/         /      |
|--__  |   -- |\________________________________/~~\~~|    /  \     /     \   |
|   |~~--__  |~_|____|____|____|____|____|____|/ /  \/|\ /      \/          \/|
|   |      |~--_|__|____|____|____|____|____|_/ /|    |/ \    /   \       /   |
|___|______|__|_||____|____|____|____|____|__[]/_|----|    \/       \  /      |
|  \mmmm :   | _|___|____|____|____|____|____|___|  /\|   /  \      /  \      |
|      B :_--~~ |_|____|____|____|____|____|____|  |  |\/      \ /        \   |
|  __--P :  |  /                                /  /  | \     /  \          /\|
|~~  |   :  | /                                 ~~~   |  \  /      \      /   |
|    |      |/                        .-.             |  /\          \  /     |
|    |      /                        |   |            |/   \          /\      |
|    |     /                        |     |            -_   \       /    \    |
+-----------------------------------------------------------------------------+
|          |  /|  |   |  2  3  4  | /~~~~~\ |       /|    |_| ....  ......... |
|          |  ~|~ | % |           | | ~J~ | |       ~|~ % |_| ....  ......... |
|   AMMO   |  HEALTH  |  5  6  7  |  \===/  |    ARMOR    |#| ....  ......... |
+-----------------------------------------------------------------------------+

In [ ]:
# --- Setup ---
!apt-get update -y
!apt-get install -y build-essential cmake git libboost-all-dev libsdl2-dev libjpeg-dev libpng-dev zlib1g-dev libpython3-dev

!apt-get install ffmpeg


import os

# Use Mesa for software rendering (CPU-based)
os.environ['LIBGL_ALWAYS_SOFTWARE'] = '1'

# Set XDG_RUNTIME_DIR to avoid other SDL issues
os.environ['XDG_RUNTIME_DIR'] = '/tmp'

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packa

In [ ]:
# --- Install ViZDoom from source (for proper linking) ---
!pip uninstall -y vizdoom
!apt-get install -y libboost-python-dev
!git clone https://github.com/mwydmuch/ViZDoom.git
%cd ViZDoom
!mkdir build && cd build
!cmake .. -DBUILD_PYTHON=ON -DCMAKE_INSTALL_PREFIX=/usr/local
!make -j$(nproc) install
%cd /content
!pip install vizdoom

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-python-dev is already the newest version (1.74.0.3ubuntu7).
libboost-python-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'ViZDoom'...
remote: Enumerating objects: 18734, done.
remote: Counting objects: 100% (567/567), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 18734 (delta 469), reused 369 (delta 369), pack-reused 18167 (from 5)
Receiving objects: 100% (18734/18734), 59.90 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (11722/11722), done.
/content/ViZDoom
CMake Warning:
  Ignoring extra path from command line:

   ".."


CMake Error: The source directory "/content" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No rule to make target 'install'.  Stop.
/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 

In [ ]:
import os
import json
import yaml
import shutil
import time
import random
import logging
import glob
import psutil
import gc
from collections import Counter
from datetime import datetime
import subprocess
from contextlib import contextmanager
from typing import Optional, Dict, Any

import cv2
import numpy as np
import pandas as pd
import imageio

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
from vizdoom import DoomGame, ScreenFormat, ScreenResolution, Mode  # Import DoomGame

# --- Create config.yaml if it doesn't exist ---
CONFIG_PATH = "config.yaml"  # Path to your config file

default_config = {
    "drive_model_dir": "/content/drive/My Drive/ViZDoomModels",
    "drive_wad_dir": "/content/drive/My Drive/ViZDoomWADs",
    "local_wad_dir": "/content/ViZDoomWADs",
    "video_dir": "/content/drive/My Drive/ViZDoomRecordings",
    "video_filename": "game_recording.mp4",
    "record_lmp": False,
    "record_video": True,
    "video_fps": 30,
    "lmp_dir": "lmp_recordings",
    "scenario_name": "defend_the_center.cfg",
    "stack_size": 4,
    "num_episodes": 10,
    "frame_skip_training": 4,
    "frame_skip_recording": 1,
    "gamma": 0.99,
    "epsilon_start": 1.0,
    "epsilon_end": 0.05,
    "epsilon_decay": 0.995,
    "model_save_freq": 10,
    "batch_size": 64,
    "memory_capacity": 10000,
    "learning_rate": 0.0001,
    "tau": 0.005,
    "n_step": 3,
    "grad_clip_norm": 1.0,
    "epsilon_decay_rate_step": 1000,
    "use_best_model_callback": True,
    "best_model_smoothing_window": 10,
    "validation_episodes": 5,
}

if not os.path.exists(CONFIG_PATH):
    with open(CONFIG_PATH, 'w') as f:
        yaml.dump(default_config, f)

# --- Configuration Loading ---
def load_config(config_path=CONFIG_PATH):
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)

CONFIG = load_config()

# --- Logging Setup ---
def setup_logger(log_dir, timestamp):
    log_file = os.path.join(log_dir, f"training_{timestamp}.log")
    logger = logging.getLogger(__name__)

    # Clear any existing handlers
    if logger.hasHandlers():
        logger.handlers.clear()

    logger.setLevel(logging.DEBUG)

    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.DEBUG)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    stream_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    return logger

# --- Google Drive Setup ---
DRIVE_MODEL_DIR = CONFIG["drive_model_dir"]
DRIVE_WAD_DIR = CONFIG["drive_wad_dir"]
LOCAL_WAD_DIR = CONFIG["local_wad_dir"]
VIDEO_DIR = CONFIG["video_dir"]
VIDEO_FILENAME = CONFIG["video_filename"]
VIDEO_PATH = os.path.join(VIDEO_DIR, VIDEO_FILENAME)

# --- Configuration Variables ---
RECORD_LMP = CONFIG["record_lmp"]
RECORD_VIDEO = CONFIG["record_video"]
VIDEO_FPS = CONFIG["video_fps"]
LMP_DIR = CONFIG["lmp_dir"]
SCENARIO_NAME = CONFIG["scenario_name"]
STACK_SIZE = CONFIG["stack_size"]
NUM_EPISODES = CONFIG["num_episodes"]
FRAME_SKIP_TRAINING = CONFIG["frame_skip_training"]
FRAME_SKIP_RECORDING = CONFIG["frame_skip_recording"]

# --- Training Parameters ---
GAMMA = CONFIG["gamma"]
EPSILON_START = CONFIG["epsilon_start"]
EPSILON_END = CONFIG["epsilon_end"]
EPSILON_DECAY = CONFIG["epsilon_decay"]
MODEL_SAVE_FREQ = CONFIG["model_save_freq"]
BATCH_SIZE = CONFIG["batch_size"]
MEMORY_CAPACITY = CONFIG["memory_capacity"]
LEARNING_RATE = CONFIG["learning_rate"]
TAU = CONFIG["tau"]
N_STEP = CONFIG["n_step"]
GRAD_CLIP_NORM = CONFIG["grad_clip_norm"]
EPSILON_DECAY_RATE_STEP = CONFIG["epsilon_decay_rate_step"]
USE_BEST_MODEL_CALLBACK = CONFIG["use_best_model_callback"]
BEST_MODEL_SMOOTHING_WINDOW = CONFIG["best_model_smoothing_window"]
VALIDATION_EPISODES = CONFIG["validation_episodes"]

# --- Setup Google Drive ---
def setup_google_drive(logger):
    drive_mounted = os.path.exists('/content/drive/My Drive')
    if not drive_mounted:
        try:
            from google.colab import drive
            drive.mount('/content/drive')
            logger.info("Google Drive mounted successfully.")
        except Exception as e:
            logger.error(f"Error mounting Google Drive: {e}")
            return False

    os.makedirs(VIDEO_DIR, exist_ok=True)
    os.makedirs(DRIVE_MODEL_DIR, exist_ok=True)
    os.makedirs(DRIVE_WAD_DIR, exist_ok=True)
    os.makedirs(LOCAL_WAD_DIR, exist_ok=True)
    logger.info("Google Drive directories created successfully.")
    return True

def copy_scenarios_to_drive(logger):
    local_scenarios_dir = "/usr/local/lib/python3.10/dist-packages/vizdoom/scenarios"
    wad_files = glob.glob(os.path.join(local_scenarios_dir, "*.wad"))
    for wad_file in wad_files:
        dest_file = os.path.join(DRIVE_WAD_DIR, os.path.basename(wad_file))
        if os.path.exists(dest_file):
            logger.info(f"Skipped existing: {os.path.basename(wad_file)}")
        else:
            try:
                shutil.copy(wad_file, dest_file)
                logger.info(f"Copied: {os.path.basename(wad_file)}")
            except Exception as e:
                logger.error(f"Error copying: {os.path.basename(wad_file)}, error: {e}")
    return True

# --- User Input for Recording ---
def get_record_choices(logger):
    logger.info(f"Record LMP: {RECORD_LMP}, Record Video: {RECORD_VIDEO}")
    return RECORD_LMP, RECORD_VIDEO

# --- User Input for Training Episodes ---
def get_num_episodes(logger):
    logger.info(f"Number of Episodes: {NUM_EPISODES}")
    return NUM_EPISODES

# --- User Input for FRAME_SKIP ---
def get_frame_skips(logger):
    FRAME_SKIP = FRAME_SKIP_TRAINING if not RECORD_VIDEO else FRAME_SKIP_RECORDING
    logger.info(f"Frame Skip Training: {FRAME_SKIP_TRAINING}, Frame Skip Recording: {FRAME_SKIP_RECORDING}")
    return FRAME_SKIP_TRAINING, FRAME_SKIP_RECORDING, FRAME_SKIP

def get_wad_choice(logger):
    print("Choose a WAD file to use:")
    print("1. Use current scenario WAD (defend_the_center.cfg with vizdoom assets)")
    print("2. Use original Doom WAD from Google Drive (ViZDoomWADs folder with defend_the_center.cfg)")
    wad_choice = input("Enter your choice (1 or 2): ")
    logger.info(f"WAD Choice: {wad_choice}")
    return wad_choice

def setup_scenario(wad_choice, logger):
    global SCENARIO_NAME
    SCENARIO_NAME = "defend_the_center.cfg"

    if wad_choice == "1":
        try:
            import vizdoom
            vizdoom_path = os.path.dirname(vizdoom.__file__)
            SCENARIO_PATH = os.path.join(vizdoom_path, "scenarios", SCENARIO_NAME)
            wad_path = None
            logger.info(f"Using default scenario path: {SCENARIO_PATH}")
            return SCENARIO_PATH, wad_path
        except Exception as e:
            logger.error(f"Error loading default scenario: {e}")
            # Fallback: Try to use a path relative to the current file
            SCENARIO_PATH = os.path.join("scenarios", SCENARIO_NAME)
            logger.warning(f"Trying fallback scenario path: {SCENARIO_PATH}")
            return SCENARIO_PATH, None

    elif wad_choice == "2":
        wad_files = glob.glob(os.path.join(DRIVE_WAD_DIR, "*.wad"))
        if not wad_files:
            logger.warning("No WAD files found in ViZDoomWADs. Using default scenario.")
            return setup_scenario("1", logger)
        print("Available WAD files in ViZDoomWADs:")
        for i, file in enumerate(wad_files):
            print(f"{i+1}. {os.path.basename(file)}")
        wad_file_choice = input("Enter the number of the WAD file to use: ")
        try:
            wad_file_choice = int(wad_file_choice)
            if 1 <= wad_file_choice <= len(wad_files):
                wad_path = wad_files[wad_file_choice - 1]
                local_wad_path = os.path.join(LOCAL_WAD_DIR, os.path.basename(wad_path))
                if not os.path.exists(local_wad_path):
                    print(f"Copying {os.path.basename(wad_path)} from Google Drive to local...")
                    try:
                        shutil.copy(wad_path, local_wad_path)
                        logger.info(f"Copied {os.path.basename(wad_path)} from Google Drive to local.")
                    except Exception as e:
                        logger.error(f"Error copying wad file to local: {e}")
                        return setup_scenario("1", logger)
                else:
                    print(f"{os.path.basename(wad_path)} already exists locally. Using local copy.")
                    logger.info(f"{os.path.basename(wad_path)} already exists locally.")
                wad_path = local_wad_path
                import vizdoom
                vizdoom_path = os.path.dirname(vizdoom.__file__)
                SCENARIO_PATH = os.path.join(vizdoom_path, "scenarios", SCENARIO_NAME)
                logger.info(f"Using WAD file: {wad_path}, scenario path: {SCENARIO_PATH}")
                return SCENARIO_PATH, wad_path
            else:
                logger.warning("Invalid WAD file choice. Using default scenario.")
                return setup_scenario("1", logger)
        except ValueError:
            logger.warning("Invalid input. Using default scenario.")
            return setup_scenario("1", logger)
    else:
        logger.warning("Invalid choice. Using default scenario.")
        return setup_scenario("1", logger)

# --- Optimized DQN for T4 GPU ---
class DQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(DQN, self).__init__()
        self.use_checkpointing = True

        # Optimized conv layers with T4-specific settings
        self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)

        # Calculate conv output size
        conv_out_size = self._get_conv_output(input_shape)

        # Fully connected layers
        self.fc1 = nn.Linear(conv_out_size, 512)
        self.fc2 = nn.Linear(512, num_actions)

        # Initialize with kaiming
        self.apply(self._init_weights)

        # Enable mixed precision for T4
        self.use_amp = True

        # T4 optimizations
        if torch.cuda.is_available():
            torch.backends.cudnn.benchmark = True
            if torch.cuda.get_device_properties(0).name == 'Tesla T4':
                torch.set_float32_matmul_precision('high')

    def _get_conv_output(self, shape):
        """Helper method to calculate conv output dimensions"""
        with torch.no_grad():
            input = torch.zeros(1, *shape)
            x = F.relu(self.bn1(self.conv1(input)))
            x = F.relu(self.bn2(self.conv2(x)))
            x = F.relu(self.bn3(self.conv3(x)))
            return x.numel() // x.shape[0]

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    def forward(self, x):
        # Use AMP context if enabled
        with torch.cuda.amp.autocast() if self.use_amp else nullcontext():
            if self.use_checkpointing and self.training:
                x = torch.utils.checkpoint.checkpoint(self.conv_forward, x, use_reentrant=False)
            else:
                x = self.conv_forward(x)
            x = self.fc_forward(x)
        return x

    def conv_forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return x.view(x.size(0), -1)

    def fc_forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class NoisyLinear(nn.Module):
    def __init__(self, in_features, out_features, std_init=0.4):
        super(NoisyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init

        self.weight_mu = nn.Parameter(torch.empty(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.empty(out_features, in_features))
        self.register_buffer('weight_epsilon', torch.empty(out_features, in_features))

        self.bias_mu = nn.Parameter(torch.empty(out_features))
        self.bias_sigma = nn.Parameter(torch.empty(out_features))
        self.register_buffer('bias_epsilon', torch.empty(out_features))

        self.reset_parameters()
        self.reset_noise()

    def reset_parameters(self):
        mu_range = 1 / math.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / math.sqrt(self.in_features))
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / math.sqrt(self.out_features))

    def reset_noise(self):
        epsilon_in = self._scale_noise(self.in_features)
        epsilon_out = self._scale_noise(self.out_features)
        self.weight_epsilon.copy_(epsilon_out.outer(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)

    def _scale_noise(self, size):
        x = torch.randn(size)
        return x.sign().mul_(x.abs().sqrt_())

    def forward(self, x):
        return F.linear(x,
                       self.weight_mu + self.weight_sigma * self.weight_epsilon,
                       self.bias_mu + self.bias_sigma * self.bias_epsilon)

class DuelingDQN(nn.Module):
    def __init__(self, input_shape, num_actions, use_noisy=True):
        super(DuelingDQN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_output(input_shape)

        # Value stream
        if use_noisy:
            self.value_fc = NoisyLinear(conv_out_size, 512)
            self.value = NoisyLinear(512, 1)
        else:
            self.value_fc = nn.Linear(conv_out_size, 512)
            self.value = nn.Linear(512, 1)

        # Advantage stream
        if use_noisy:
            self.advantage_fc = NoisyLinear(conv_out_size, 512)
            self.advantage = NoisyLinear(512, num_actions)
        else:
            self.advantage_fc = nn.Linear(conv_out_size, 512)
            self.advantage = nn.Linear(512, num_actions)

        self.use_noisy = use_noisy
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    def _get_conv_output(self, shape):
        o = torch.zeros(1, *shape)
        o = F.relu(self.conv1(o))
        o = F.relu(self.conv2(o))
        o = F.relu(self.conv3(o))
        return int(np.prod(o.size()))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)

        value = F.relu(self.value_fc(x))
        value = self.value(value)

        advantage = F.relu(self.advantage_fc(x))
        advantage = self.advantage(advantage)

        return value + advantage - advantage.mean(dim=1, keepdim=True)

    def reset_noise(self):
        if self.use_noisy:
            for m in self.modules():
                if isinstance(m, NoisyLinear):
                    m.reset_noise()

# --- Prioritized Experience Replay Memory ---
class PrioritizedReplayMemory:
    def __init__(self, capacity, alpha=0.6, beta_start=0.4, beta_frames=100000):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.alpha = alpha
        self.beta_start = beta_start
        self.beta_frames = beta_frames
        self.frame = 0

    def push(self, state, action, reward, next_state, done):
        max_priority = self.priorities.max() if self.memory else 1.0

        if len(self.memory) < self.capacity:
            self.memory.append(None)

        self.memory[self.position] = (state, action, reward, next_state, done)
        self.priorities[self.position] = max_priority
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        priorities = self.priorities[:len(self.memory)]
        probabilities = priorities**self.alpha
        probabilities /= probabilities.sum()

        indices = np.random.choice(len(self.memory), batch_size, p=probabilities)
        samples = [self.memory[idx] for idx in indices]

        beta = min(1.0, self.beta_start + (self.frame * (1 - self.beta_start) / self.beta_frames))
        weights = (len(self.memory) * probabilities[indices])**(-beta)
        weights /= weights.max()
        self.frame += 1

        return samples, indices, weights

    def update_priorities(self, indices, td_errors):
        self.priorities[indices] = np.abs(td_errors) + 1e-6

    def __len__(self):
        return len(self.memory)

# --- N-step Buffer ---
class NStepBuffer:
    def __init__(self, n_step):
        self.n_step = n_step
        self.buffer = []

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def get(self):
        if len(self.buffer) < self.n_step:
            return None

        n_step_state, n_step_action, n_step_reward, n_step_next_state, n_step_done = self.buffer[0]
        for i in range(1, self.n_step):
            state, action, reward, next_state, done = self.buffer[i]
            n_step_reward += reward * GAMMA**i
            n_step_next_state = next_state
            n_step_done = done or n_step_done

        self.buffer = self.buffer[1:]
        return n_step_state, n_step_action, n_step_reward, n_step_next_state, n_step_done

    def __len__(self):
        return len(self.buffer)

# --- DQNAgent ---
class DQNAgent:
    def __init__(self, state_shape, num_actions, learning_rate, gamma, epsilon_start, epsilon_end, epsilon_decay, memory_capacity, batch_size, tau, n_step, grad_clip_norm, epsilon_decay_rate_step):
        self.state_shape = state_shape
        self.num_actions = num_actions
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.tau = tau
        self.grad_clip_norm = grad_clip_norm
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = DQN(state_shape, num_actions).to(self.device)
        self.target_net = DQN(state_shape, num_actions).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.memory = PrioritizedReplayMemory(memory_capacity)
        self.n_step_buffer = NStepBuffer(n_step)
        self.epsilon_decay_rate_step = epsilon_decay_rate_step
        self.training_step = 0
        self.best_avg_reward = float('-inf')
        self.best_model_state = None

    def select_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.num_actions)
        else:
            with torch.no_grad():
                state = torch.tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
                q_values = self.policy_net(state)
                return q_values.argmax(dim=1).item()

    def update_epsilon(self):
        self.epsilon = max(self.epsilon_end, self.epsilon_start * (self.epsilon_decay ** (self.training_step / self.epsilon_decay_rate_step)))

    def learn(self):
        if len(self.memory) < self.batch_size:
            return None, None, None
        self.training_step += 1
        samples, indices, weights = self.memory.sample(self.batch_size)
        batch = tuple(zip(*samples))

        state_batch = torch.tensor(np.array(batch[0]), dtype=torch.float32, device=self.device)
        action_batch = torch.tensor(batch[1], dtype=torch.long, device=self.device).unsqueeze(1)
        reward_batch = torch.tensor(batch[2], dtype=torch.float32, device=self.device).unsqueeze(1)
        next_state_batch = torch.tensor(np.array(batch[3]), dtype=torch.float32, device=self.device)
        done_batch = torch.tensor(batch[4], dtype=torch.float32, device=self.device).unsqueeze(1)
        weights = torch.tensor(weights, dtype=torch.float32, device=self.device).unsqueeze(1)

        q_values = self.policy_net(state_batch).gather(1, action_batch)

        with torch.no_grad():
            next_q_values_online = self.policy_net(next_state_batch)
            next_actions = next_q_values_online.argmax(dim=1).unsqueeze(1)
            next_q_values_target = self.target_net(next_state_batch).gather(1, next_actions)
            expected_q_values = reward_batch + self.gamma * next_q_values_target * (1 - done_batch)

        td_errors = expected_q_values - q_values
        loss = (weights * F.smooth_l1_loss(q_values, expected_q_values, reduction='none')).mean()

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), self.grad_clip_norm)
        self.optimizer.step()

        self.memory.update_priorities(indices, td_errors.detach().cpu().numpy().squeeze())
        return loss.item(), q_values.mean().item(), torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), self.grad_clip_norm).item()

    def update_target_network(self):
        for target_param, local_param in zip(self.target_net.parameters(), self.policy_net.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)

    def save_model(self, model_path):
        torch.save({
            'policy_net_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, model_path)

    def load_model(self, model_path):
        checkpoint = torch.load(model_path, map_location=self.device)
        self.policy_net.load_state_dict(checkpoint['policy_net_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.target_net.load_state_dict(self.policy_net.state_dict())

def scan_for_models(agent, model_dir, logger):
    model_files = glob.glob(os.path.join(model_dir, "*.pth"))

    if not model_files:
        logger.info("No existing models found. Creating new model...")
        return agent  # Return the existing agent instead of creating new one

    print("Available models:")
    for i, file in enumerate(model_files):
        print(f"{i+1}. {os.path.basename(file)}")
    print(f"{len(model_files)+1}. Create new model")

    choice = input("Enter your choice: ")
    try:
        choice = int(choice)
        if choice in range(1, len(model_files) + 2):
            if choice <= len(model_files):
                model_path = model_files[choice - 1]
                try:
                    agent.load_model(model_path)
                    logger.info(f"Model loaded from {model_path}")
                    return agent
                except Exception as e:
                    logger.error(f"Error loading model: {e}")
                    return agent  # Return existing agent on error
            else:
                logger.info("Creating new model...")
                return agent  # Return existing agent for new model
        else:
            logger.warning("Invalid choice. Using existing model.")
            return agent
    except ValueError:
        logger.warning("Invalid input. Using existing model.")
        return agent

def setup_vizdoom(scenario_path, wad_path=None, logger=None):
    try:
        from vizdoom import DoomGame, ScreenFormat, ScreenResolution, Mode
        game = DoomGame()

        if scenario_path:
            game.load_config(scenario_path)
        else:
            raise ValueError("Scenario path is not defined.")

        if wad_path:
            game.set_doom_game_path(wad_path)
        game.set_window_visible(False)
        game.set_screen_format(ScreenFormat.RGB24)
        game.set_screen_resolution(ScreenResolution.RES_320X240)

        if RECORD_LMP:
            game.set_mode(Mode.PLAYER)
            os.makedirs(LMP_DIR, exist_ok=True)

        game.init()
        num_actions = game.get_available_buttons_size()
        actions = np.identity(num_actions, dtype=int).tolist()

        screen_height, screen_width = game.get_screen_height(), game.get_screen_width()
        channels = game.get_screen_channels()
        state_shape = (STACK_SIZE * channels, 84, 84)
        logger.info("ViZDoom initialized successfully.")
        return game, actions, state_shape
    except Exception as e:
        if logger:
            logger.error(f"Error during ViZDoom setup: {e}")
        print(f"Error during ViZDoom setup: {e}")
        return None, None, None

# --- Video Writer ---
def setup_video_writer(video_dir, video_fps, logger=None):
    """Setup video writer with timestamped filename"""
    if RECORD_VIDEO:
        try:
            # Create timestamp prefix
            timestamp = datetime.now().strftime("%y%m%d%H%M")
            video_filename = f"{timestamp}_game_recording.mp4"
            video_path = os.path.join(video_dir, video_filename)

            # Create video writer
            video_writer = imageio.get_writer(video_path, fps=video_fps)
            if logger:
                logger.info(f"Video Writer setup at: {video_path}")
            return video_writer, video_path

        except Exception as e:
            if logger:
                logger.error(f"Error during video writer setup: {e}")
            return None, None
    return None, None

@contextmanager
def video_writer_context(video_dir, video_fps, logger):
    """Context manager for video writer"""
    writer = None
    path = None
    try:
        writer, path = setup_video_writer(video_dir, video_fps, logger)
        yield writer, path
    finally:
        if writer:
            try:
                writer.close()
                if logger and path:
                    logger.info(f"Video saved to: {path}")
            except Exception as e:
                if logger:
                    logger.error(f"Error closing video writer: {e}")

# --- TensorBoard Setup ---
def setup_tensorboard(log_dir, logger=None):
    now = datetime.now()
    dt_string = now.strftime("%Y%m%d%H%M")
    log_dir = os.path.join(log_dir, f"{dt_string}_experiment")
    if logger:
      logger.info(f"TensorBoard logging to: {log_dir}")
    return SummaryWriter(log_dir=log_dir)

# --- Frame Stacking ---
def preprocess_frame(frames):
    if len(frames) == 0:
        return []
    processed_frames = []
    for frame in frames:
        if len(frame.shape) == 3 and frame.shape[0] == 3:
            frame = np.mean(frame, axis=0)
        frame = frame.astype(np.float32) / 255.0
        frame = cv2.resize(frame, (84, 84), interpolation=cv2.INTER_AREA)
        if len(frame.shape) == 3:
            frame = frame.transpose(2, 0, 1)
        else:
            frame = np.expand_dims(frame, axis=0)
        processed_frames.append(frame)
    return processed_frames

def create_stacked_state(state_buffer):
    processed_frames = preprocess_frame(state_buffer)
    if any(f is None for f in processed_frames) or not all(f.shape == processed_frames[0].shape for f in processed_frames):
        print("Error: Inconsistent or None frames in processed_frames")
        return None
    stacked_state = np.concatenate(processed_frames, axis=0)
    return stacked_state

def get_game_state_info(game):
    game_state = game.get_state()
    if game_state is not None:
        damage_taken = game_state.game_variables[0]
        damage_inflicted = game_state.game_variables[1]
        return damage_taken, damage_inflicted
    else:
        return 0, 0

def run_episode(agent, game, actions, episode, frame_skip, lmp_dir, record_video, video_writer, stack_size, state_shape, logger, episode_metrics_df):
    with resource_cleanup():
        if RECORD_LMP:
            lmp_file_path = os.path.join(lmp_dir, f"episode_{episode + 1}.lmp")
            game.new_episode(lmp_file_path)
        else:
            game.new_episode()

        game_state = game.get_state()
        if game_state is None or game_state.screen_buffer is None:
            logger.error(f"Skipping episode {episode+1} due to invalid game state.")
            return None, 0, 0, 0, 0, 0, None

        state_buffer = [game_state.screen_buffer] * stack_size
        state = create_stacked_state(state_buffer)
        if state is None:
            logger.error(f"Skipping episode {episode + 1} due to state initialization error.")
            return None, 0, 0, 0, 0, 0, None

        total_reward = 0
        step_count = 0
        episode_start_time = time.time()
        damage_taken = 0
        damage_inflicted = 0
        action_counts = Counter()
        inference_times = []
        loss, avg_q_value, grad_norm = None, None, None

        while not game.is_episode_finished():
            inference_start_time = time.time()

            action_index = agent.select_action(state)

            inference_end_time = time.time()
            inference_times.append(inference_end_time - inference_start_time)

            action = actions[action_index]
            action_counts[action_index] += 1

            reward = game.make_action(action, frame_skip)
            done = game.is_episode_finished()

            if not done:
                game_state = game.get_state()
                if game_state and game_state.screen_buffer is not None:
                    next_frame = game_state.screen_buffer
                    state_buffer.pop(0)
                    state_buffer.append(next_frame)
                    next_state = create_stacked_state(state_buffer)

                    if next_state is None:
                        logger.error(f"Error: next_state is None in episode {episode + 1} at step {step_count + 1}.")
                        break
                    damage_taken_step, damage_inflicted_step = get_game_state_info(game)
                    damage_taken += damage_taken_step
                    damage_inflicted += damage_inflicted_step
                else:
                    logger.error(f"Error: Invalid state in episode {episode+1}, step {step_count+1}.")
                    next_state = np.zeros(state_shape)
                    break
            else:
                next_state = np.zeros(state_shape)

            agent.n_step_buffer.push(state, action_index, reward, next_state, done)

            n_step_experience = agent.n_step_buffer.get()
            if n_step_experience:
                n_step_state, n_step_action, n_step_reward, n_step_next_state, n_step_done = n_step_experience
                agent.memory.push(n_step_state, n_step_action, n_step_reward, n_step_next_state, n_step_done)

            loss_step, avg_q_value_step, grad_norm_step = agent.learn()
            if loss_step is not None:
                loss = loss_step
                avg_q_value = avg_q_value_step
                grad_norm = grad_norm_step

            state = next_state
            total_reward += reward
            step_count += 1

            if record_video and not done:
                most_recent_frame = state_buffer[-1]
                try:
                    # Add frame counter overlay
                    frame_counter = f"Episode {episode+1} Step {step_count}"
                    cv2.putText(most_recent_frame, frame_counter, (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

                    video_writer.append_data(most_recent_frame)
                except Exception as e:
                    logger.error(f"Error appending frame to video: {e}")
                    break

        episode_survival_time = time.time() - episode_start_time
        total_actions = sum(action_counts.values())
        if total_actions > 0:
            action_diversity = sum(count / total_actions for count in action_counts.values()) / len(actions)
        else:
            action_diversity = 0

        avg_inference_time = np.mean(inference_times) if inference_times else 0

        if total_reward is not None:
            new_row = {
                'episode': episode + 1,
                'reward': total_reward,
                'steps': step_count,
                'survival_time': episode_survival_time,
                'damage_taken': damage_taken,
                'damage_inflicted': damage_inflicted,
                'action_diversity': action_diversity,
                'avg_inference_time': avg_inference_time,
                'loss': loss if loss is not None else np.nan,
                'avg_q_value': avg_q_value if avg_q_value is not None else np.nan,
                'grad_norm': grad_norm if grad_norm is not None else np.nan
            }
            return total_reward, step_count, episode_survival_time, damage_taken, damage_inflicted, action_diversity, avg_inference_time, new_row
        else:
            return None, 0, 0, 0, 0, 0, 0, None

    # After each episode
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# --- Validation ---
def validate_model(agent, game, actions, state_shape, stack_size, num_episodes, logger):
    """Validates the model by running a number of episodes and calculating the mean reward."""
    total_rewards = []
    for episode in range(num_episodes):
        total_reward, _, _, _, _, _, _, _ = run_episode(agent, game, actions, episode, FRAME_SKIP_RECORDING, None, False, None, stack_size, state_shape, logger, pd.DataFrame())
        if total_reward is not None:
            total_rewards.append(total_reward)
    return np.mean(total_rewards) if total_rewards else float('-inf')

def best_model_callback(agent, episode_rewards, logger, model_dir, best_model_smoothing_window):
    """Callback to save the best model based on a smoothed average reward."""
    if len(episode_rewards) >= best_model_smoothing_window:
        avg_reward = np.mean(episode_rewards[-best_model_smoothing_window:])
    else:
        avg_reward = np.mean(episode_rewards) if episode_rewards else float('-inf')
    if avg_reward > agent.best_avg_reward:
        agent.best_avg_reward = avg_reward
        agent.best_model_state = deepcopy(agent.policy_net.state_dict())
        model_filename = f"best_dqn_model.pth"
        model_path = os.path.join(model_dir, model_filename)
        agent.save_model(model_path)
        logger.info(f"Best model saved to {model_path} with reward: {agent.best_avg_reward}")

# --- Save metrics to CSV ---
def save_metrics(episode_rewards, episode_lengths, episode_survival_times,
                 episode_damage_taken, episode_damage_inflicted, logger, episode_metrics_df, training_time,
                 ram_usage_list, gpu_memory_usage_list, action_diversity_list):
    metrics_df = pd.DataFrame({
    'episode': range(1, len(episode_rewards) + 1),
    'reward': episode_rewards,
    'steps': episode_lengths,
    'epsilon': [EPSILON_START * (EPSILON_DECAY ** (i / EPSILON_DECAY_RATE_STEP)) for i in range(len(episode_rewards))],
    'training_time': training_time,
    'survival_time': episode_survival_times,
    'damage_taken': episode_damage_taken,
    'damage_inflicted': episode_damage_inflicted,
        'ram_usage': ram_usage_list,
        'gpu_memory_usage': gpu_memory_usage_list,
        'action_diversity': action_diversity_list,
    'model_save_freq': MODEL_SAVE_FREQ,
    'batch_size': BATCH_SIZE,
    'memory_capacity': MEMORY_CAPACITY,
    'learning_rate': LEARNING_RATE,
    'tau': TAU,
    'n_step': N_STEP
    })
    # Merge episode metrics with overall metrics
    metrics_df = pd.merge(metrics_df, episode_metrics_df, on='episode', how='left')

    now = datetime.now()
    dt_string = now.strftime("%Y%m%d%H%M")
    metrics_filename = f"{dt_string}_training_metrics.csv"
    metrics_path = os.path.join(DRIVE_MODEL_DIR, metrics_filename)
    metrics_df.to_csv(metrics_path, index=False)
    logger.info(f"Training metrics saved to {metrics_path}")

# --- Main Training Loop ---
def cleanup_resources():
    # Clear memory
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def close_writers(video_writer=None, tensorboard_writer=None):
    try:
        if video_writer:
            video_writer.close()  # Use close() instead of release() for imageio writer
        if tensorboard_writer:
            tensorboard_writer.close()
    except Exception as e:
        print(f"Error closing writers: {e}")

def validate_input(value, param_type, min_val, max_val, default):
    """Validate and convert user input"""
    if not value:  # Handle empty input
        return default

    try:
        if param_type == "float":
            val = float(value)
        else:
            val = int(value)

        if min_val <= val <= max_val:
            return val

        print(f"Value {val} out of range ({min_val}-{max_val}). Using default: {default}")
        return default

    except ValueError:
        print(f"Invalid input '{value}'. Using default: {default}")
        return default

def get_hyperparameters():
    params = {
        "learning_rate": {"default": 0.00025, "type": "float", "min": 0, "max": 1,
                         "desc": "Step size for optimizer (0-1)"},
        "batch_size": {"default": 64, "type": "int", "min": 32, "max": 512,
                      "desc": "Batch size for training (32-512)"},
        "memory_capacity": {"default": 50000, "type": "int", "min": 10000, "max": 1000000,
                          "desc": "Max experiences in memory (10k-1M)"},
        "gamma": {"default": 0.99, "type": "float", "min": 0, "max": 1,
                 "desc": "Discount factor (0-1)"},
        "tau": {"default": 0.005, "type": "float", "min": 0, "max": 1,
                "desc": "Target network update rate (0-1)"},
        "epsilon_start": {"default": 1.0, "type": "float", "min": 0, "max": 1,
                         "desc": "Initial exploration rate (0-1)"},
        "epsilon_end": {"default": 0.05, "type": "float", "min": 0, "max": 1,
                       "desc": "Final exploration rate (0-1)"},
        "epsilon_decay": {"default": 0.995, "type": "float", "min": 0, "max": 1,
                         "desc": "Exploration decay rate (0-1)"},
        "epsilon_decay_rate_step": {"default": 5000, "type": "int", "min": 1000, "max": 100000,
                                   "desc": "Steps for epsilon decay (1k-100k)"},
        "n_step": {"default": 3, "type": "int", "min": 1, "max": 10,
                   "desc": "Steps for n-step learning (1-10)"},
        "grad_clip_norm": {"default": 1.0, "type": "float", "min": 0, "max": 10,
                          "desc": "Gradient clipping norm (0-10)"},
        "frame_stack_size": {"default": 4, "type": "int", "min": 2, "max": 4,
                            "desc": "Number of stacked frames (2-4)"}
    }

    print("\nHyperparameter Configuration Menu")
    print("=================================")

    for param_name, param_info in params.items():
        print(f"\n{param_name}: {param_info['desc']}")
        print(f"Default: {param_info['default']}")
        user_input = input(f"Enter value (or press Enter for default): ").strip()

        if user_input:
            params[param_name]["value"] = validate_input(
                user_input,
                param_info["type"],
                param_info["min"],
                param_info["max"],
                param_info["default"]
            )
        else:
            params[param_name]["value"] = param_info["default"]

    return params

class ResourceMonitor:
    def __init__(self, logger, use_gpu: bool = False):
        self.logger = logger
        self.use_gpu = use_gpu and torch.cuda.is_available()
        self.process = psutil.Process()
        if self.use_gpu:
            self.log_gpu_info()

    def log_gpu_info(self):
        """Log GPU device information at startup"""
        if self.use_gpu:
            device = torch.cuda.get_device_properties(0)
            self.logger.info(f"GPU Device: {device.name}")
            self.logger.info(f"Total GPU Memory: {device.total_memory / 1024**2:.2f} MB")
            try:
                nvidia_smi = subprocess.check_output(["nvidia-smi"], encoding='utf-8')
                self.logger.info(f"NVIDIA-SMI Output:\n{nvidia_smi}")
            except:
                self.logger.warning("Could not get nvidia-smi output")

    def get_system_metrics(self) -> Dict[str, Any]:
        """Collect system resource metrics"""
        metrics = {
            'process_ram_mb': self.process.memory_info().rss / 1024**2,
            'system_ram_percent': psutil.virtual_memory().percent,
            'available_ram_mb': psutil.virtual_memory().available / 1024**2,
            'cpu_percent': psutil.cpu_percent(),
            'per_cpu_percent': psutil.cpu_percent(percpu=True)
        }

        if self.use_gpu:
            metrics.update({
                'gpu_allocated_mb': torch.cuda.memory_allocated() / 1024**2,
                'gpu_reserved_mb': torch.cuda.memory_reserved() / 1024**2,
                'gpu_max_allocated_mb': torch.cuda.max_memory_allocated() / 1024**2
            })
            try:
                nvidia_out = subprocess.check_output(
                    ['nvidia-smi', '--query-gpu=utilization.gpu,temperature.gpu',
                     '--format=csv,noheader,nounits'],
                    encoding='utf-8'
                )
                gpu_util, gpu_temp = map(int, nvidia_out.strip().split(','))
                metrics.update({
                    'gpu_utilization': gpu_util,
                    'gpu_temperature': gpu_temp
                })
            except:
                self.logger.warning("Could not get NVIDIA-SMI metrics")

        return metrics

    def log_metrics(self, tensorboard_writer, episode):
        """Log metrics to both logger and tensorboard"""
        metrics = self.get_system_metrics()

        # Log to tensorboard
        for name, value in metrics.items():
            if not isinstance(value, (list, tuple)):
                tensorboard_writer.add_scalar(f"Resources/{name}", value, episode)

        # Format metrics for logger
        metrics_str = ", ".join([
            f"{k}: {v:.2f}" if isinstance(v, float)
            else f"{k}: {v}" for k, v in metrics.items()
            if not isinstance(v, (list, tuple))
        ])
        self.logger.info(f"Resource Usage - {metrics_str}")

        return metrics

@contextmanager
def resource_cleanup(game=None, video_writer=None, tensorboard_writer=None, agent=None):
    """Context manager for proper resource cleanup"""
    try:
        yield
    finally:
        try:
            if video_writer:
                video_writer.close()
            if tensorboard_writer:
                tensorboard_writer.close()
            if game:
                game.close()
            if agent:
                # Explicitly delete large objects
                del agent.policy_net
                del agent.target_net
                del agent.memory
                del agent

            # Clear CUDA cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Force garbage collection
            gc.collect()

        except Exception as e:
            print(f"Error during cleanup: {e}")

# --- Video Recording Configuration ---
def get_video_config():
    """Get video recording configuration from user"""
    try:
        record_video = input("\nEnable video recording? (y/n, default=n): ").lower().strip() == 'y'
        frame_skip = FRAME_SKIP_RECORDING if record_video else FRAME_SKIP_TRAINING
        video_fps = VIDEO_FPS if record_video else None

        if record_video:
            frame_skip_input = input("Enter frame skip rate (1-10, default=1): ").strip()
            try:
                frame_skip = int(frame_skip_input) if frame_skip_input else 1
                frame_skip = max(1, min(10, frame_skip))
            except ValueError:
                frame_skip = 1
                print("Invalid input. Using default frame skip of 1")

        return {
            'record_video': record_video,
            'frame_skip': frame_skip,
            'video_fps': video_fps
        }
    except Exception as e:
        print(f"Error in video config: {e}. Using defaults.")
        return {
            'record_video': False,
            'frame_skip': FRAME_SKIP_TRAINING,
            'video_fps': None
        }

# --- Video Writer Context Manager ---
@contextmanager
def video_writer_context(video_dir, video_fps, logger):
    """Context manager for video writer with timestamped filename"""
    writer = None
    path = None
    try:
        if RECORD_VIDEO:
            timestamp = datetime.now().strftime("%y%m%d%H%M")
            video_filename = f"{timestamp}_game_recording.mp4"
            video_path = os.path.join(video_dir, video_filename)
            writer = imageio.get_writer(video_path, fps=video_fps)
            logger.info(f"Video recording started: {video_path}")
            path = video_path
        yield writer, path
    finally:
        if writer:
            try:
                writer.close()
                if path:
                    logger.info(f"Video saved: {path}")
            except Exception as e:
                logger.error(f"Error closing video writer: {e}")

def main():
    try:
        # Initialize components
        TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M")
        LOG_DIR = "logs"
        os.makedirs(LOG_DIR, exist_ok=True)
        logger = setup_logger(LOG_DIR, TIMESTAMP)

        # Get video configuration once
        video_config = get_video_config()
        global RECORD_VIDEO, FRAME_SKIP, VIDEO_FPS
        RECORD_VIDEO = video_config['record_video']
        FRAME_SKIP = video_config['frame_skip']
        VIDEO_FPS = video_config['video_fps']

        # Setup environment
        setup_google_drive(logger)
        copy_scenarios_to_drive(logger)
        wad_choice = get_wad_choice(logger)
        SCENARIO_PATH, wad_path = setup_scenario(wad_choice, logger)
        game, actions, state_shape = setup_vizdoom(SCENARIO_PATH, wad_path, logger)
        tensorboard_writer = setup_tensorboard("runs", logger)

        # Get hyperparameters from user or use defaults
        params = get_hyperparameters()

        # Initialize agent with proper parameters
        agent = DQNAgent(
            state_shape=state_shape,
            num_actions=len(actions),
            learning_rate=params["learning_rate"]["value"],
            gamma=params["gamma"]["value"],
            epsilon_start=params["epsilon_start"]["value"],
            epsilon_end=params["epsilon_end"]["value"],
            epsilon_decay=params["epsilon_decay"]["value"],
            memory_capacity=params["memory_capacity"]["value"],
            batch_size=params["batch_size"]["value"],
            tau=params["tau"]["value"],
            n_step=params["n_step"]["value"],
            grad_clip_norm=params["grad_clip_norm"]["value"],
            epsilon_decay_rate_step=params["epsilon_decay_rate_step"]["value"]
        )

        # Initialize metrics DataFrame with proper columns
        episode_metrics_df = pd.DataFrame(columns=[
            'episode', 'reward', 'steps', 'survival_time',
            'damage_taken', 'damage_inflicted', 'action_diversity',
            'avg_inference_time', 'loss', 'avg_q_value', 'grad_norm'
        ])

        # Training loop with proper resource management
        with resource_cleanup(game, None, tensorboard_writer, agent):
            for episode in range(NUM_EPISODES):
                with video_writer_context(VIDEO_DIR, VIDEO_FPS, logger) as (video_writer, video_path):
                    (total_reward, step_count, episode_survival_time,
                     damage_taken, damage_inflicted, action_diversity,
                     avg_inference_time, new_row) = run_episode(
                        agent, game, actions, episode, FRAME_SKIP,
                        LMP_DIR, RECORD_VIDEO, video_writer,
                        STACK_SIZE, state_shape, logger, episode_metrics_df)

                    if total_reward is not None:
                        # Update metrics
                        if new_row is not None:
                            episode_metrics_df = pd.concat([
                                episode_metrics_df,
                                pd.DataFrame([new_row])
                            ], ignore_index=True)

                        # Save model periodically
                        if episode % MODEL_SAVE_FREQ == 0:
                            model_path = os.path.join(
                                DRIVE_MODEL_DIR,
                                f"dqn_model_episode_{episode}.pth"
                            )
                            agent.save_model(model_path)
                            logger.info(f"Model saved to {model_path}")

                        # Log episode results
                        logger.info(
                            f"Episode {episode + 1}: "
                            f"Reward = {total_reward:.2f}, "
                            f"Steps = {step_count}, "
                            f"Damage Taken = {damage_taken}, "
                            f"Damage Inflicted = {damage_inflicted}"
                        )

    except Exception as e:
        logger.error(f"Error in main: {e}")
        raise
    finally:
        cleanup_resources()

if __name__ == "__main__":
    main()


Enable video recording? (y/n, default=n): y
Enter frame skip rate (1-10, default=1): 


2025-01-06 23:24:30,492 - INFO - Google Drive directories created successfully.
INFO:__main__:Google Drive directories created successfully.
2025-01-06 23:24:30,497 - INFO - Skipped existing: simpler_basic.wad
INFO:__main__:Skipped existing: simpler_basic.wad
2025-01-06 23:24:30,502 - INFO - Skipped existing: rocket_basic.wad
INFO:__main__:Skipped existing: rocket_basic.wad
2025-01-06 23:24:30,506 - INFO - Skipped existing: predict_position.wad
INFO:__main__:Skipped existing: predict_position.wad
2025-01-06 23:24:30,509 - INFO - Skipped existing: defend_the_line.wad
INFO:__main__:Skipped existing: defend_the_line.wad
2025-01-06 23:24:30,512 - INFO - Skipped existing: cig.wad
INFO:__main__:Skipped existing: cig.wad
2025-01-06 23:24:30,514 - INFO - Skipped existing: deathmatch.wad
INFO:__main__:Skipped existing: deathmatch.wad
2025-01-06 23:24:30,516 - INFO - Skipped existing: cig_with_unknown.wad
INFO:__main__:Skipped existing: cig_with_unknown.wad
2025-01-06 23:24:30,518 - INFO - Skipp

Choose a WAD file to use:
1. Use current scenario WAD (defend_the_center.cfg with vizdoom assets)
2. Use original Doom WAD from Google Drive (ViZDoomWADs folder with defend_the_center.cfg)
Enter your choice (1 or 2): 


2025-01-06 23:24:32,588 - INFO - WAD Choice: 
INFO:__main__:WAD Choice: 
2025-01-06 23:24:32,591 - WARNING - Invalid choice. Using default scenario.
2025-01-06 23:24:32,593 - INFO - Using default scenario path: /usr/local/lib/python3.10/dist-packages/vizdoom/scenarios/defend_the_center.cfg
INFO:__main__:Using default scenario path: /usr/local/lib/python3.10/dist-packages/vizdoom/scenarios/defend_the_center.cfg
2025-01-06 23:24:33,010 - INFO - ViZDoom initialized successfully.
INFO:__main__:ViZDoom initialized successfully.
2025-01-06 23:24:33,015 - INFO - TensorBoard logging to: runs/202501062324_experiment
INFO:__main__:TensorBoard logging to: runs/202501062324_experiment



Hyperparameter Configuration Menu

learning_rate: Step size for optimizer (0-1)
Default: 0.00025
Enter value (or press Enter for default): 

batch_size: Batch size for training (32-512)
Default: 64
Enter value (or press Enter for default): 

memory_capacity: Max experiences in memory (10k-1M)
Default: 50000
Enter value (or press Enter for default): 

gamma: Discount factor (0-1)
Default: 0.99
Enter value (or press Enter for default): 

tau: Target network update rate (0-1)
Default: 0.005
Enter value (or press Enter for default): 

epsilon_start: Initial exploration rate (0-1)
Default: 1.0
Enter value (or press Enter for default): 

epsilon_end: Final exploration rate (0-1)
Default: 0.05
Enter value (or press Enter for default): 

epsilon_decay: Exploration decay rate (0-1)
Default: 0.995
Enter value (or press Enter for default): 

epsilon_decay_rate_step: Steps for epsilon decay (1k-100k)
Default: 5000
Enter value (or press Enter for default): 

n_step: Steps for n-step learning (1-10

2025-01-06 23:24:52,530 - INFO - Video recording started: /content/drive/My Drive/ViZDoomRecordings/2501062324_game_recording.mp4
INFO:__main__:Video recording started: /content/drive/My Drive/ViZDoomRecordings/2501062324_game_recording.mp4
<ipython-input-7-1b8ab9d2b91b>:302: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast() if self.use_amp else nullcontext():
<ipython-input-7-1b8ab9d2b91b>:1221: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  episode_metrics_df = pd.concat([
2025-01-06 23:25:00,077 - INFO - Model saved to /content/drive/My Drive/ViZDoomModels/dqn_model_episode_0.pth
INFO:__main__:Model saved to /content/drive/My Drive/Vi

SyntaxError: incomplete input (<ipython-input-15-8a532c2ea7f5>, line 1394)